In [1]:
from pulp import *
import csv
import numpy as np
import pandas as pd
import os
import time
import collections
from tqdm import tqdm
os.chdir('/home/valesco/Datasets/')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [2]:
nfl_df = pd.read_csv('2015_NFL_DK')

In [3]:
nfl_df.head()

,home_away,opponent,player_first,player_last,full_name,total_points,position,salary,team,week
0,home,nor,"Palmer,",Carson,"Palmer,Carson",28.68,QB,6500,ari,week1
1,home,pit,"Brady,",Tom,"Brady,Tom",27.62,QB,7700,nwe,week1
2,home,nyg,"Romo,",Tony,"Romo,Tony",27.14,QB,7300,dal,week1
3,away,tam,"Mariota,",Marcus,"Mariota,Marcus",24.96,QB,6000,ten,week1
4,home,det,"Rivers,",Philip,"Rivers,Philip",24.96,QB,7100,sdg,week1


In [4]:
in_file = "2015_NFL_DK"
reader = csv.DictReader(open(in_file, 'r'))
player_dict_pre = []
for i , row in enumerate(reader):
    team = {k: v for k, v in row.items()}
    team['id'] = i
    player_dict_pre.append(team)
    
players = [player['id'] for player in player_dict_pre]
teams = [0]

In [5]:
num_teams = 1000
row_count = 0
opt_df = []
opt_df = pd.DataFrame(columns = ['qb_name', 'qb_pos', 'qb_points', 'qb_salary',
                                 'wr1_name', 'wr1_pos', 'wr1_points', 'wr1_salary',
                                 'wr2_name', 'wr2_pos', 'wr2_points', 'wr2_salary',
                                 'wr3_name', 'wr3_pos', 'wr3_points', 'wr3_salary',
                                 'rb1_name', 'rb1_pos', 'rb1_points', 'rb1_salary',
                                 'rb2_name', 'rb2_pos', 'rb2_points', 'rb2_salary',
                                 'te_name', 'te_pos', 'te_points', 'te_salary',
                                 'flex_name', 'flex_pos', 'flex_points', 'flex_salary',
                                 'def_name', 'def_pos', 'def_points', 'def_salary',
                                 'team_points', 'total_salary', 'season_year', 'week'
                                ])

time_ls = ['week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8', 'week9',
           'week10', 'week11', 'week12', 'week13', 'week14', 'week15', 'week16', 'week17']

#player_dict = []
#for player in player_dict_pre:
#    if player['season_year'] == '2015' and player['week'] == '17':
#        player_dict.append(player)

master_ls = {}
        
for time in tqdm(time_ls):
    for i, t1 in enumerate(range(num_teams)):
        team_dict = {}
        
        player_dict = []
        for player in player_dict_pre:
            if player['week'] == time:
                player_dict.append(player)
        
        
        if i == 0:
            # Set objective
            players = [player['id'] for player in player_dict]            
            
            opt = LpProblem('Optimizer', LpMaximize)
            possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

            #objective
            opt += sum([float(player['total_points']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])

            #set constraints
            for team in teams:
                opt += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 9, '6 players per team '+str(team)
                opt += lpSum([float(player['salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
                opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'QB']) == 1, 'one QB per team '
                opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'WR']) >= 3, 'two or more WR per team '
                opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'RB']) >= 2, 'two or more RB per team '
                opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'TE']) >= 1, 'two or more TE per team '
                opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'DEF']) == 1, 'two or more DEF per team '
                #opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['time_line'] == time]) == 9, 'same week'
                
            opt.writeLP('fantasy_optimization.lp')
            opt.solve()
            status = LpStatus[opt.status]
            selected = []
            

            for player in player_dict:
                if possible_assignments[0][player['id']].value() == 1.0:
                    players = {}
                    #print(player['player'] + ' salary: ' + player['salary'] + ' proj: ' + player['odds'])
                    #opt_odds.append(float(player['odds']))
                    name = player['full_name']
                    players['full_name'] = player['full_name']
                    players['position'] = player['position']
                    players['total_points'] = player['total_points']
                    players['year'] = '2015'
                    players['week'] = player['week']
                    players['salary'] = player['salary']
                    #players['time_line'] = player['time_line']
                    selected.append(player['id'])
                    team_dict[player['id']] = players
                    
            master_ls[i] = team_dict

        else:
            # Set objective
            players = [player['id'] for player in player_dict]
            
            opt = LpProblem('Optimizer', LpMaximize)
            possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

            #objective
            opt += sum([float(player['total_points']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])
            
            c = 0
            for s in range(int(len(selected)/6)):
                opt += lpSum(possible_assignments[0][selected[c]] + possible_assignments[0][selected[c+1]] +
                              possible_assignments[0][selected[c+2]] + possible_assignments[0][selected[c+3]] +
                              possible_assignments[0][selected[c+4]] + possible_assignments[0][selected[c+5]]) <= 5, 'selected' + str(s)
                c += 6

            #set constraints
            
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 9, '6 players per team '+str(team)
            opt += lpSum([float(player['salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'QB']) == 1, 'one QB per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'WR']) >= 3, 'two or more WR per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'RB']) >= 2, 'two or more RB per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'TE']) >= 1, 'two or more TE per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'DEF']) == 1, 'two or more DEF per team '
            #opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['time_line'] == time]) == 9, 'same week'
                
            opt.writeLP('fantasy_optimization.lp')
            opt.solve()
            status = LpStatus[opt.status]
            

            for player in player_dict:
                if possible_assignments[0][player['id']].value() == 1.0:
                    players = {}
                    #print(player['player'] + ' salary: ' + player['salary'] + ' proj: ' + player['odds'])
                    #opt_odds.append(float(player['odds']))
                    name = player['full_name']
                    players['full_name'] = player['full_name']
                    players['position'] = player['position']
                    players['total_points'] = player['total_points']
                    players['season_year'] = '2015'
                    players['week'] = player['week']
                    players['salary'] = player['salary']
                    #players['time_line'] = player['time_line']
                    selected.append(player['id'])
                    team_dict[player['id']] = players
            
            master_ls[i] = team_dict
            
    for k, v in master_ls.items():
        points = 0
        salary = 0
        wr_count = 0
        rb_count = 0
        te_count = 0
        for k1, v1 in master_ls[k].items(): 
            points += float(v1['total_points'])
            salary += float(v1['salary'])
            if v1['position'] == 'QB':
                qb_name = v1['full_name']
                qb_pos = 'QB'
                qb_points = v1['total_points']
                qb_salary = v1['salary']
                #year = v1['season_year']
                year = '2015'
                week = v1['week']
                #timeline = v1['time_line']
            elif v1['position'] == 'WR':
                if wr_count == 0:
                    wr1_name = v1['full_name']
                    wr1_pos = 'WR1'
                    wr1_points = v1['total_points']
                    wr1_salary = v1['salary']
                elif wr_count == 1:
                    wr2_name = v1['full_name']
                    wr2_pos = 'WR2'
                    wr2_points = v1['total_points']
                    wr2_salary = v1['salary']
                elif wr_count == 2:
                    wr3_name = v1['full_name']
                    wr3_pos = 'WR3'
                    wr3_points = v1['total_points']
                    wr3_salary = v1['salary']
                else:
                    flex_name = v1['full_name']
                    flex_pos = 'flex_WR'
                    flex_points = v1['total_points']
                    flex_salary = v1['salary']
                wr_count += 1
            elif v1['position'] == 'RB':
                if rb_count == 0:
                    rb1_name = v1['full_name']
                    rb1_pos = 'RB1'
                    rb1_points = v1['total_points']
                    rb1_salary = v1['salary']
                elif rb_count == 1:
                    rb2_name = v1['full_name']
                    rb2_pos = 'RB2'
                    rb2_points = v1['total_points']
                    rb2_salary = v1['salary']
                else:
                    flex_name = v1['full_name']
                    flex_pos = 'flex_RB'
                    flex_points = v1['total_points']
                    flex_salary = v1['salary']
                rb_count += 1
            elif v1['position'] == 'TE':
                if te_count == 0:
                    te_name = v1['full_name']
                    te_pos = 'TE'
                    te_points = v1['total_points']
                    te_salary = v1['salary']
                else:
                    flex_name = v1['full_name']
                    flex_pos = 'flex_TE'
                    flex_points = v1['total_points']
                    flex_salary = v1['salary']
                te_count += 1
            elif v1['position'] == 'DEF':
                def_name = v1['full_name']
                def_pos = 'DEF'
                def_points = v1['total_points']
                def_salary = v1['salary']
            else:
                print('ERROR!!!')


        opt_df.loc[row_count] = [qb_name, qb_pos, qb_points, qb_salary, wr1_name, wr1_pos, wr1_points, wr1_salary,
                                 wr2_name, wr2_pos, wr2_points, wr2_salary, wr3_name, wr3_pos, wr3_points, wr3_salary,
                                 rb1_name, rb1_pos, rb1_points, rb1_salary, rb2_name, rb2_pos, rb2_points, rb2_salary,
                                te_name, te_pos, te_points, te_salary, flex_name, flex_pos, flex_points, flex_salary,
                                 def_name, def_pos, def_points, def_salary, points, salary, year, week]
        
        row_count += 1

  0%|          | 0/17 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
opt_df

In [7]:
opt_df.to_pickle('2015_optimal_lineups_1k.p')

In [17]:
opt_df['flex_pos'].describe()

count       17000
unique          3
top       flex_WR
freq         8257
Name: flex_pos, dtype: object

In [18]:
from bokeh.io import output_notebook, show
from bokeh.charts import Scatter
from bokeh.charts import Line
from bokeh.charts import Histogram
output_notebook()

Loading BokehJS ...

In [5]:
player_dict_pre[0]

{'full_name': 'Palmer,Carson',
 'home_away': 'home',
 'id': 0,
 'opponent': 'nor',
 'player_first': 'Palmer,',
 'player_last': 'Carson',
 'position': 'QB',
 'salary': '6500',
 'team': 'ari',
 'total_points': '28.68',
 'week': 'week1'}

In [38]:
num_df['flex_points'] = pd.to_numeric(opt_df['flex_points'], errors = 'coerce')
num_df.groupby('flex_pos').mean()

,qb_points,qb_salary,flex_points,flex_salary,team_points,total_salary
flex_pos,,,,,,
flex_RB,30.118374,6144.382979,30.118374,30.118374,273.479957,46314.787234
flex_TE,34.189523,6322.631709,34.189523,34.189523,272.502765,47810.883008
flex_WR,34.472930,6359.283033,34.472930,34.472930,282.115885,47353.651447


In [32]:
num_df.dtypes

qb_name          object
qb_pos           object
qb_points        object
qb_salary         int64
wr1_name         object
wr1_pos          object
wr1_points       object
wr1_salary       object
wr2_name         object
wr2_pos          object
wr2_points       object
wr2_salary       object
wr3_name         object
wr3_pos          object
wr3_points       object
wr3_salary       object
rb1_name         object
rb1_pos          object
rb1_points       object
rb1_salary       object
rb2_name         object
rb2_pos          object
rb2_points       object
rb2_salary       object
te_name          object
te_pos           object
te_points        object
te_salary        object
flex_name        object
flex_pos         object
flex_points      object
flex_salary      object
def_name         object
def_pos          object
def_points       object
def_salary       object
team_points     float64
total_salary    float64
season_year      object
week             object
dtype: object